In [1]:
import porespy as ps
import numpy as np

import supervisely_lib as sly
import os

In [2]:
# Change this field to the name of your team, where target workspace exists.
team_name = "marcwang17" # Automatically inserted

# Change this field to the of your workspace, where target project exists.
workspace_name = "First Workspace" # Automatically inserted

# Change this field to the name of your target project.
project_name = "split_dt_ups" # Automatically inserted

agent_name = "Dazzling Duck"

model_name = 'particle_unet'

# If you are running this notebook on a Supervisely web instance, the connection
# Edit those values if you run this notebook on your own PC
# details below will be filled in from environment variables automatically.
#
# If you are running this notebook locally on your own machine, edit to fill in the
# connection details manually. You can find your access token at
# "Your name on the top right" -> "Account settings" -> "API token".
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']

In [3]:
# Initialize API object
api = sly.Api(address, token)

In [4]:
team = api.team.get_info_by_name(team_name)
if team is None:
    raise RuntimeError("Team {!r} not found".format(team_name))

workspace = api.workspace.get_info_by_name(team.id, workspace_name)
if workspace is None:
    raise RuntimeError("Workspace {!r} not found".format(workspace_name))
    
project = api.project.get_info_by_name(workspace.id, project_name)
if project is None:
    raise RuntimeError("Project {!r} not found".format(project_name))

agent = api.agent.get_info_by_name(team.id, agent_name)
if agent is None:
    raise RuntimeError("Agent {!r} not found".format(agent_name))
if agent.status is api.agent.Status.WAITING:
    raise RuntimeError("Agent {!r} is not running".format(agent_name))
    
print("Team: id={}, name={}".format(team.id, team.name))
print("Workspace: id={}, name={}".format(workspace.id, workspace.name))
print("Project: id={}, name={}".format(project.id, project.name))
print("Agent: id={}, name={}".format(agent.id, agent.name))

RuntimeError: Project 'split_dt_ups' not found

In [14]:
inference_project_name = "{}_inference".format(project.name)

inf_project = api.project.get_info_by_name(workspace.id, inference_project_name)
if project is None:
    inf_project = api.project.create(workspace.id, inference_project_name, change_name_if_conflict=True)

In [15]:
inference_config = {
  "model": {
    "gpu_device": 0
  },
  "mode": {
    "name": "full_image",
    "model_classes": {
      "add_suffix": "_unet"
    }
  }
}


In [16]:
model_info = api.model.get_info_by_name(workspace.id, model_name) 

In [17]:
task_id = api.task.run_inference(
            agent.id, project.id, model_info.id, inference_project_name, inference_config=inference_config)

In [18]:
task_id

161693